In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import gc
import os
import sys
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/train_V2.csv')
test_data = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/test_V2.csv')

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
train_data.head()

In [ ]:
train_data.columns

In [ ]:
train_data.describe()

In [ ]:
train_data.info()

In [ ]:
train_data.isna().any()

In [ ]:
null_columns = pd.DataFrame({'Columns': train_data.isna().sum().index,'No. Null values': train_data.isna().sum().values, 'Percentage': train_data.isna().sum().values/train_data.shape[0]})

In [ ]:
null_columns

# find the uniqueID

In [ ]:
train_data['Id'].nunique()

# find the groupID

In [ ]:
train_data['groupId'].nunique()

# find matchID

In [ ]:
train_data['matchId'].nunique()

In [ ]:
train_data['matchType']

In [ ]:
train_data.groupby(['matchType']).count()

In [ ]:
plt.figure(figsize = (9,5))
train_data.groupby('matchId')['matchType'].first().value_counts().plot.bar()

# Map the match

In [ ]:
new_train_data = train_data
def mapthematch(data):
    mapping = lambda y: 'solo' if ('solo' in y) else 'duo' if('duo' in y) or ('crash' in y) else 'squad'
    data['matchType'] = data['matchType'].apply(mapping)
    return (new_train_data)
data = mapthematch(new_train_data)
data.groupby('matchId')['matchType'].first().value_counts().plot.bar()

# Groups count per matchID

In [ ]:
matchwise_group_count = data[['matchType', 'matchId','groupId']]
a = matchwise_group_count.groupby(['matchId', 'matchType']).count()
print(matchwise_group_count.nunique())
a

# player per each group

In [ ]:
groups = data.groupby(['matchType', 'matchId', 'groupId']).size().to_frame('players in each group')
groups

# find the illegal match

In [ ]:
data[data['winPlacePerc'].isnull()]

In [ ]:
data.drop(2744604, inplace = True)
data[data['winPlacePerc'].isnull()]

# visualise the data

In [ ]:
a

In [ ]:
a.groupby('matchType').hist(bins = 70)

# Match Duration

In [ ]:
data['matchDuration'].hist(bins = 40)

# minimum match duration

In [ ]:
data['matchDuration'].min()

# maximum match duration

In [ ]:
data['matchDuration'].max()

# Team kills based on matchtype

In [ ]:
d = data[['teamKills','matchType']]
d.groupby('matchType').hist(bins = 50)

# NORMALISE THE KILL COLUMN

In [ ]:
data['killsNormalization'] = data['kills']*((100-data['kills'])/100 +1)

# Normalise the damageDealt column

In [ ]:
data['damageDealtNormalization'] = data['damageDealt']*((100- data['damageDealt'])/100 +1)

# Normalise the maxplace column

In [ ]:
data['maxPlaceNormalization'] = data['maxPlace']*((100- data['maxPlace'])/100 + 1)

# Normalise the matchDuration column

In [ ]:
data['matchDurationNormalization'] = data['matchDuration']*((100- data['matchDuration'])/100 + 1)

# Visualise the new normalise columns and the previous one to analyse it

In [ ]:
New_normalised_column = data[['Id', 'matchDuration','matchDurationNormalization','kills','killsNormalization','damageDealt','damageDealtNormalization','maxPlace','maxPlaceNormalization','matchDuration','matchDurationNormalization']]

In [ ]:
New_normalised_column

# creating new features

In [ ]:
data['healsandboostsfeature']= data['heals']+ data['boosts']
data[['heals','boosts','healsandboostsfeature']].tail()

# Total distance travelled

In [ ]:
data['totalDistancetravelled'] = data['rideDistance']+ data['walkDistance']+ data['swimDistance']
data['totalDistancetravelled']

In [ ]:
data['headshot_rate'] = data['headshotKills']/data['kills']
data['headshot_rate']

In [ ]:
data

# DATA OUTLIER

In [ ]:
data['killswithoutMovinganytime'] = ((data['kills']>0)&(data['totalDistancetravelled']==0))

In [ ]:
data['killswithoutMovinganytime'] 

# CHECK PLAYER WHO KILL WITHOUT MOVING

In [ ]:
data[data['killswithoutMovinganytime'] == True].shape

In [ ]:
data[data['killswithoutMovinganytime'] == True].head()

In [ ]:
#remove outliers
data.drop(data[data['killswithoutMovinganytime']==True].index, inplace = True)

# Visualise the longest skill and try to find what are th outlier we have

In [ ]:
import seaborn as sns
plt.figure(figsize=(12,4))
sns.distplot(data['longestKill'])
plt.show()

# check out players who made kills with a distance of more than 1km

In [ ]:
display(data[data['longestKill']>=900].shape)
data[data['longestKill']>=900].head(10)

In [ ]:
data.shape


In [ ]:
data['winPlacePerc']

In [ ]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression

In [ ]:
data.columns

In [ ]:
x =  data[['killsNormalization', 'damageDealtNormalization','maxPlaceNormalization','matchDurationNormalization','healsandboostsfeature','totalDistancetravelled']]

In [ ]:
y = data['winPlacePerc']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state =42)

In [ ]:
l= LinearRegression()
l.fit(x_train,y_train)

In [ ]:
y_pred = l.predict(x_test)

In [ ]:
y_pred

In [ ]:
print(l.intercept_)

In [ ]:
print(l.coef_)

In [ ]:
df = pd.DataFrame({'Actual': y_test,'Predicted': y_pred})
df

In [ ]:
df1 = df.head(25)
df1.plot(kind = 'bar', figsize = (20,10))
plt.grid(which = 'major', linestyle= '-', linewidth = '0.5',color = 'green')
plt.grid(which = 'minor', linestyle = ':', linewidth = '0.5', color = 'black')
plt.show()

In [ ]:
print('Mean Absolute Error: ', metrics.mean_absolute_error(y_test,y_pred))
print('Mean Squared Error: ', metrics.mean_squared_error(y_test,y_pred))
print('Root Mean Mean Error: ', np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
